In [1]:
%reload_ext autoreload
%autoreload 2

# Imports

In [2]:
from kret_notebook import *  # NOTE import first
from kret_lgbm._core.lgbm_nb_imports import *
from kret_lightning._core.lightning_nb_imports import *
from kret_matplotlib._core.mpl_nb_imports import *
from kret_np_pd._core.np_pd_nb_imports import *
from kret_optuna._core.optuna_nb_imports import *
from kret_polars._core.polars_nb_imports import *
from kret_rosetta._core.rosetta_nb_imports import *
from kret_sklearn._core.sklearn_nb_imports import *
from kret_torch_utils._core.torch_nb_imports import *
from kret_tqdm._core.tqdm_nb_imports import *
from kret_type_hints._core.types_nb_imports import *
from kret_utils._core.utils_nb_imports import *

# from kret_wandb._core.wandb_nb_imports import *  # NOTE this is slow to import

[kret_np_pd._core.np_pd_nb_imports] Imported kret_np_pd._core.np_pd_nb_imports in 0.0267 seconds
Loaded environment variables from /Users/Akseldkw/coding/kretsinger/.env
[kret_lgbm._core.lgbm_nb_imports] Imported kret_lgbm._core.lgbm_nb_imports in 0.0009 seconds
[kret_lightning._core.lightning_nb_imports] Imported kret_lightning._core.lightning_nb_imports in 0.0062 seconds
[kret_matplotlib._core.mpl_nb_imports] Imported kret_matplotlib._core.mpl_nb_imports in 0.1351 seconds
[kret_optuna._core.optuna_nb_imports] Imported kret_optuna._core.optuna_nb_imports in 0.0002 seconds
[kret_polars._core.polars_nb_imports] Imported kret_polars._core.polars_nb_imports in 1.2356 seconds
[kret_rosetta._core.rosetta_nb_imports] Imported kret_rosetta._core.rosetta_nb_imports in 0.0000 seconds
[kret_sklearn._core.sklearn_nb_imports] Imported kret_sklearn._core.sklearn_nb_imports in 0.2742 seconds
[kret_torch_utils._core.torch_nb_imports] Imported kret_torch_utils._core.torch_nb_imports in 1.0781 seconds


In [3]:
from kret_lightning.examples.cifar10_module import *
from kret_lightning.examples.cifar10_datamodule import *

# Load Data

In [4]:
cifar_dm = CIFAR10DataModule(UKS_CONSTANTS.DATA_DIR / "CIFAR10", batch_size=64, num_workers=6)

In [5]:
cifar_dm.data_dir

PosixPath('/Users/Akseldkw/coding/data_kretsinger/CIFAR10')

# Implementation

In [ ]:
cifar_nn = CIFAR10ResNet(lr=0.001)
cifar_nn.hparams_initial

Saving hparams, ignoring ()
Saving hparams, ignoring ()


"dropout_rate":     0.3
"l1_penalty":       0.0
"l2_penalty":       0.0
"lr":               0.001
"num_blocks":       2
"num_classes":      10
"num_filters":      64
"patience":         10
"warmup_step_frac": 0.1

In [7]:
cifar_nn.save_load_logging_dict

{'save_dir': PosixPath('/Users/Akseldkw/coding/data_kretsinger/lightning_logs'),
 'name': 'CIFAR10ResNet',
 'version': 'v_001'}

In [ ]:
static_args = TrainerStaticDefaults.OPTUNA_SWEEP.copy()
# static_args["max_epochs"] = 10
static_args

{'max_epochs': 50,
 'limit_train_batches': None,
 'limit_val_batches': None,
 'log_every_n_steps': 20,
 'enable_model_summary': False,
 'enable_checkpointing': False,
 'max_time': {'hours': 2}}

In [24]:
dynamic_args = TrainerDynamicDefaults.trainer_dynamic_defaults(cifar_nn, cifar_dm, logtype=True)
dynamic_args

{'logger': None,
 'default_root_dir': PosixPath('/Users/Akseldkw/coding/data_kretsinger/lightning_logs/CIFAR10ResNet/v_001'),
 'callbacks': []}

In [25]:
# dynamic_args["logger"].save_dir, dynamic_args["logger"].name

In [26]:
trainer_args = static_args | dynamic_args

In [27]:
trainer = L.Trainer(**trainer_args)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores


In [ ]:
def objective(trial: optuna.trial.Trial) -> float:
    # import traceback

    # try:

    # preset = trial.suggest_categorical("preset", ["tiny", "small", "medium", "large", "xlarge"])
    num_blocks = trial.suggest_int("num_blocks", 1, 4)  # 1=tiny, 2=small, 3=medium, 4=large
    num_filters = trial.suggest_categorical("num_filters", [32, 64, 128])
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5)
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    l1 = trial.suggest_float("l1", 1e-5, 1e-2, log=True)
    l2 = trial.suggest_float("l2", 1e-5, 1e-2, log=True)

    model = CIFAR10ResNet(
        num_blocks=num_blocks,
        num_filters=num_filters,
        dropout_rate=dropout_rate,
        lr=lr,
        l1_penalty=l1,
        l2_penalty=l2,
        patience=4,
    )
    model._sweep_mode = True  # NOTE important!
    dynamic_args = TrainerDynamicDefaults.trainer_dynamic_defaults(model, cifar_dm, logtype=None, trial=trial)
    trainer_args = static_args | dynamic_args

    trainer = L.Trainer(**trainer_args)  # New trainer per trial!
    assert trainer.logger is not None
    trainer.logger.log_hyperparams(model.hparams_initial)
    trainer.fit(model, datamodule=cifar_dm, **TrainerStaticDefaults.TRAINER_FIT)

    return trainer.callback_metrics["val_f1"].item()


# except Exception as e:
#     print(f"\n{'='*80}")
#     print(f"Exception in trial {trial.number}:")
#     print(f"{'='*80}")
#     traceback.print_exc()
#     print(f"{'='*80}\n")
#     raise  # Re-raise so Optuna knows the trial failed

In [30]:
OptunaDefaults.CREATE_STUDY_DEFAULTS

{'pruner': <optuna.pruners._hyperband.HyperbandPruner at 0x17db2e120>,
 'load_if_exists': True,
 'storage': 'sqlite://///Users/Akseldkw/coding/data_kretsinger/optuna_studies/optuna_studies.db'}

In [ ]:
pruner = optuna.pruners.HyperbandPruner()

study = optuna.create_study(
    study_name="cifar10_resnet_finetune",
    direction="maximize",
    **OptunaDefaults.CREATE_STUDY_DEFAULTS,
)

[I 2026-01-22 22:05:32,518] Using an existing study with name 'cifar10_resnet_v2' instead of creating a new one.


In [32]:
optim_args = OptunaDefaults.study_n_hours(3) | OptunaDefaults.OPTIM_STUDY_DEF
# optim_args["n_jobs"] = 1
optim_args

{'n_trials': None,
 'timeout': 10800,
 'n_jobs': 1,
 'gc_after_trial': True,
 'show_progress_bar': True}

In [33]:
study.optimize(objective, **optim_args)

   0%|          | 00:00/3:00:00

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores


Saving hparams, ignoring ()
Saving hparams, ignoring ()


Loading `train_dataloader` to estimate number of stepping batches.


Output()

...

Output()

Metric val_loss improved. New best score: 1.537



Detected KeyboardInterrupt, attempting graceful shutdown ...


SystemExit: 1

/Users/Akseldkw/micromamba/envs/kret_312/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3709: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
print(next(cifar_nn.parameters()).dtype), print(next(cifar_nn.parameters()).device)

torch.float32
mps:0


(None, None)

# Evaluate

## Load

In [ ]:
study_names = optuna.study.get_all_study_names(storage=UKS_CONSTANTS.OPTUNA_STORAGE_DB)
study_names

['cifar10_resnet', 'cifar10_resnet_v1', 'cifar10_resnet_v2']

In [ ]:
study_summaries = optuna.study.get_all_study_summaries(storage=UKS_CONSTANTS.OPTUNA_STORAGE_DB)

In [ ]:
studyload = study_summaries[0]

In [ ]:
studyload.n_trials

77

In [ ]:
studyload.datetime_start

datetime.datetime(2026, 1, 19, 11, 4, 24, 454090)

In [ ]:
study.best_trial.params

{'num_blocks': 2,
 'num_filters': 128,
 'dropout_rate': 0.32117446313532205,
 'lr': 0.0003469745658031787,
 'l1': 5.1683361683017804e-05,
 'l2': 8.742339664966662e-05}

In [ ]:
out = optuna.importance.get_param_importances(study)

In [ ]:
out

{'num_filters': np.float64(0.3417231459629884),
 'l2': np.float64(0.253645835286912),
 'dropout_rate': np.float64(0.20035574159074557),
 'lr': np.float64(0.13402967229963758),
 'l1': np.float64(0.04068302844741069),
 'num_blocks': np.float64(0.029562576412305672)}

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
# Plot parameter importances
optuna.visualization.plot_param_importances(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
UKS_TH_UTILS.func_to_typed_dict(optuna.visualization.plot_optimization_history)

from typing import TypedDict

class Plot_optimization_history_TypedDict(TypedDict, total=False):
    study: 'Study | Sequence[Study]'
    target: 'Callable[[FrozenTrial], float] | None'  # = None
    target_name: 'str'  # = 'Objective Value'
    error_bar: 'bool'  # = False


In [ ]:
study.n_trials

AttributeError: 'Study' object has no attribute 'n_trials'